In [39]:
using LinearAlgebra
using Test

## 1

### RESOLVE_DIAGONAL

In [40]:
function resolve_diagonal(A, b)
    n = length(b)
    x = zeros(n)
    for i=1:n
        x[i] = b[i] / A[i,i]
    end
    return x
end

resolve_diagonal (generic function with 1 method)

#### Testes

In [41]:
n = 5
A1 = randn()*Matrix{Float64}(I,n,n)
b1 = randn(n)

A2 = randn()*Matrix{Float64}(I,n,n)
b2 = randn(n)

A3 = randn()*Matrix{Float64}(I,n,n)
b3 = randn(n)

5-element Vector{Float64}:
 -0.4204119472456362
  1.3420391534929759
  0.32051382320709887
  0.5960390138657149
  0.7662247466485679

In [42]:
@test resolve_diagonal(A1,b1) ≈ A1\b1 

Test Passed

In [43]:
@test resolve_diagonal(A2,b2) ≈ A2\b2 

Test Passed

In [44]:
@test resolve_diagonal(A3,b3) ≈ A3\b3 

Test Passed

### RESOLVE_TRIANGULAR_SUPERIOR

In [45]:
function resolve_triangular_superior(A, b)
    n = length(b)
    x = zeros(n)
    for i = n:-1:1
        x[i] = b[i]
        for j = i+1:n
            x[i] = x[i] - (A[i,j]*x[j])
        end
        x[i] = x[i]/A[i, i]
    end
    return x
end

resolve_triangular_superior (generic function with 1 method)

#### Testes

In [46]:
function resolve_triangular_inferior(A, b)
    n = length(b)
    x = zeros(n)
    for i = 1:n
        # Vetor com os resultados
        x[i] = b[i]
        for j = 1:(i - 1)
            x[i] = x[i] - (A[i,j]*x[j])
        end
        x[i] = x[i]/A[i,i]
    end
    return x
end

resolve_triangular_inferior (generic function with 1 method)

In [47]:
function decomposicao_LU(A)
    # Pega o número de linhas de A
    n, = size(A)
    # Gera a matriz que será L 
    L = Matrix{Float64}(I,n,n)
    # Fazemos uma cópia de A. Será nossa matriz triangular superior
    U = copy(A)
    # Constante que multiplica as linhas
    v = 0
    for i = 1:(n-1)
        pivo = U[i,i]
        # Para cada linha abaixo da linha do pivo, realizamos o escalonamento
        for j = (i+1):n 
            # Constante que multiplicará toda uma linha
            v = U[j,i]/pivo
            # Multiplica todos elementos da linha correspondente para balancear o sistema
            for k = 1:n  
                U[j,k] = U[j,k] - U[i,k]*v
            end
            # Zeramos os elementos manualmente devido ao erro de precisão da máquina
            U[j,i] = 0 
            # Gera a matriz triangular inferior elemento por elemento
            L[j,i] = v
        end
    end
    return L,U
end         

decomposicao_LU (generic function with 1 method)

In [48]:
A =37*randn(5,5)
L,U = decomposicao_LU(A)
L*U ≈ A 
#U
L

5×5 Matrix{Float64}:
  1.0       0.0         0.0        0.0       0.0
  2.58733   1.0         0.0        0.0       0.0
 -4.19842  -0.119406    1.0        0.0       0.0
  4.68228  -0.0857574  -0.784976   1.0       0.0
  6.41157   0.569692   -1.42186   -0.318857  1.0

In [49]:
A =37*randn(5,5)

println(A[:,1])
println(A)

[-24.429936047813655, -18.874540636369883, -4.339049298228314, 6.202539904809807, 19.20919018610366]
[-24.429936047813655 28.754439375214986 30.420106170732524 25.053466218864646 -14.588037734788376; -18.874540636369883 -45.100858566470215 -4.2305589452484265 -16.794245115842156 61.679972087231235; -4.339049298228314 7.886276974853253 -7.820829472268075 6.194604398525881 2.953225396355694; 6.202539904809807 52.741040629250556 40.29845825349469 -43.244740010822646 -47.6833315271298; 19.20919018610366 10.383743334453214 -31.32838675171 34.26960457785082 19.776079138501814]


In [50]:
function resolve(A,b)
    L,U = decomposicao_LU(A)
    Y = resolve_triangular_inferior(L,b)
    return resolve_triangular_superior(U,Y)
end

resolve (generic function with 1 method)

In [60]:
A = randn(n,n)
b = A * ones(n)
@test resolve(A,b) ≈ A\b

Test Passed

In [62]:
A = randn(n,n)

5×5 Matrix{Float64}:
  0.929597   0.685575   0.664074    0.0934643  -1.12662
 -0.13872   -0.080687   1.68679     0.258155    0.223977
  0.880768   0.270508   3.10584     1.08844    -0.214803
 -1.10138   -0.285054  -0.0938987  -0.349839    0.981171
  0.439383   1.07314   -0.815929    0.665115    0.467584

In [64]:
A[:,1] = A[:,5]
A

5×5 Matrix{Float64}:
 -1.12662    0.685575   0.664074    0.0934643  -1.12662
  0.223977  -0.080687   1.68679     0.258155    0.223977
 -0.214803   0.270508   3.10584     1.08844    -0.214803
  0.981171  -0.285054  -0.0938987  -0.349839    0.981171
  0.467584   1.07314   -0.815929    0.665115    0.467584

In [66]:
function inversa(A)
    n, = size(A)
    A_inv = zeros(n, n)
    Ident = Matrix{Float64}(I,n,n)
    for i=1:n
        A_inv[:,i] = resolve(A,Ident[:,i])
    end
    return A_inv
end               

inversa (generic function with 1 method)

In [77]:
A = randn(n,n)
@test inversa(A) ≈ inv(A)

Test Passed

## 4